Magnetoencephalography (MEG)
----------

<table style="margin-left: 0px">
  <tr>
    <td>
      <img width=1000 height=200 src="MEG_experiment.PNG"/>
    </td>
   </tr>
</table>


In [1]:
# %install mne 0.15.dev0

In [15]:
import numpy as np
from IPython.display import display
import mne
mne.set_log_level('WARNING')

import bqplot.pyplot as plt

### Loading data using `MNE`

In [3]:
from mne.datasets import spm_face
data_path = spm_face.data_path()  # downloaded automatically (approx. 2GB)
raw_fname = data_path + '/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds'
print(raw_fname)

/Users/maded/mne_data/MNE-spm-face/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds


In [4]:
raw = mne.io.read_raw_ctf(raw_fname, preload=True)
print(raw)

<RawCTF  |  SPM_CTF_MEG_example_faces1_3D.meg4, n_channels x n_times : 340 x 324474 (676.0 sec), ~842.5 MB, data loaded>


In [5]:
raw.filter(1., 40., fir_design='firwin')

<RawCTF  |  SPM_CTF_MEG_example_faces1_3D.meg4, n_channels x n_times : 340 x 324474 (676.0 sec), ~842.5 MB, data loaded>

In [6]:
events = mne.find_events(raw, stim_channel='UPPT001', verbose=True)

print(events[:5])

172 events found
Events id: [1 2 3]
[[2026    0    3]
 [3759    0    3]
 [5484    0    3]
 [7226    0    3]
 [8951    0    1]]


In [7]:
event_id = {"faces": 1, "scrambled": 2}
tmin, tmax = -0.1, 0.5  # start and end of an epoch in sec.

# Set up indices of channels to include in analysis
picks = mne.pick_types(raw.info, meg=True, stim=True, eog=True,
                       ref_meg=True, exclude='bads')

# Read epochs
decim = 2  # decimate to make the example faster to run
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=None, preload=True,
                    reject=dict(mag=1.5e-12), decim=decim)

print(epochs)

<Epochs  |  n_events : 167 (all good), tmin : -0.1 (s), tmax : 0.5 (s), baseline : None, ~57.1 MB, data loaded,
 'faces': 84, 'scrambled': 83>


In [8]:
# %matplotlib inline
evoked = epochs.average()
# evoked.plot_joint(times=[0.105, 0.130, 0.180]);

In [9]:
data = epochs.get_data()
data_mean = np.mean(data, axis=0)[31:]

In [10]:
# Color Utilities

def _rgb(x, y, z):
    """Transform x, y, z values into RGB colors."""
    rgb = np.array([x, y, z]).T
    rgb -= rgb.min(0)
    rgb /= rgb.max(0)
    return rgb

def rgb_to_css(rgb):
    '''
    Converts rgb color representation to a css hex
    '''
    for color in rgb:
        if color > 1 or color < 0:
            raise ValueError('rgb values must be between 0 and 1, a value of rgb({}) was given'.format(rgb))
    hexes = ["%02X" % int(color*255) for color in rgb]
    return ''.join(['#'] + hexes)


def color_scale(data, colors):
    n_colors = len(colors)
    mi, ma = data.min(), data.max()
    def f(x):
        index = int(n_colors * (x - mi) / (ma - mi))
        return colors[index - 1]
    return f

def diverging_color_scale(data, colors):
    mi, ma = data.min(), data.max()
    def f(x):
        if x < 0:
            t = (mi - x) / mi
            t = np.clip(t, 0, 1)
            return (1-t) * colors[0] + t * colors[1]
        else:
            t = (ma - x) / ma
            t = np.clip(t, 0, 1)
            return (1-t) * colors[2] + t * colors[1]
    return f

In [11]:
chs = evoked.info['chs']
locs3d = np.array([ch['loc'][:3] for ch in chs])[31:]
x, y, z = locs3d.T
colors = _rgb(x, y, z)
colors = [rgb_to_css(c) for c in colors]

In [12]:
from bqplot import ColorScale

In [13]:
fig2 = plt.figure(min_aspect_ratio=1, max_aspect_ratio=1, title='Seen from above',
                  layout={'min_width': '600px', 'min_height': '600px'})
sc_c = ColorScale(colors=['orangered', 'black', 'deepskyblue'], min=data_mean.min()* 1e15, max=data_mean.max()* 1e15)
scat = plt.scatter(x, y, colors=colors, scales={'color': sc_c},
                   axes_options={'x': {'visible': False}, 'y': {'visible': False}})
plt.show()

VBox(children=(Figure(axes=[ColorAxis(scale=ColorScale(colors=['orangered', 'black', 'deepskyblue'], max=277.79314281954834, min=-295.89795148495813)), Axis(scale=LinearScale(), visible=False), Axis(orientation='vertical', scale=LinearScale(), visible=False)], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_height='600px', min_width='600px'), marks=[Scatter(colors=['#48AFDC', '#33A3D3', '#2395C8', '#1884BB', '#1372A5', '#5BA3EA', '#459AE4', '#338DDC', '#267DD4', '#1D6BC1', '#3B7AE7', '#2B67DA', '#4E87EF', '#4065EC', '#729FF0', '#658DF4', '#5272F5', '#545BF5', '#5F48F2', '#717BFC', '#6668FC', '#7254FB', '#66FC58', '#4EF455', '#39E84D', '#29D93F', '#72FC76', '#5BF775', '#45EF72', '#31E16A', '#24D15C', '#66F592', '#4FEF8F', '#39E38A', '#29D581', '#1EC374', '#72EDAC', '#5BEAAB', '#43E0A7', '#32D3A1', '#21C493', '#16B080', '#6AE0C4', '#51D9C0', '#3CCEBB', '#2BC0B2', '#1BB09F', '#109D8B', '#5FCDD2', '#4AC4D1', '#37B7CA', '#26ABBD', '#179DAC', '#0F8C9D', '#0D7989', '#64087C', '#4C0F7B', '#361B79', '#242B78', '#700261', '#580860', '#40125E', '#2D205D', '#640344', '#4C0B43', '#361742', '#242841', '#700129', '#411026', '#2D1E26', '#64020B', '#4C090A', '#37160A', '#6A36E7', '#474AE9', '#7226D3', '#5238DF', '#3453DC', '#671BBB', '#5527CB', '#3E32CC', '#2C43C9', '#2357C6', '#4F1BAF', '#3C26B0', '#2934AF', '#1D48AF', '#175DAB', '#720D9B', '#591196', '#431995', '#2E2695', '#1F3792', '#144D92', '#10628E', '#16B460', '#0FA06B', '#0C8B75', '#0C6C71', '#0B5575', '#173E76', '#1CC249', '#0FA44B', '#098F56', '#047958', '#025D58', '#094659', '#1D335B', '#1EC92B', '#15B334', '#099437', '#027E39', '#01673B', '#04503D', '#133A3F', '#16BA16', '#0EA31F', '#02881C', '#01711E', '#015A20', '#084323', '#1D3125', '#0FA900', '#069300', '#007C01', '#006403', '#034D06', '#143808', '#24260A', '#8AC3DE', '#9FB7E1', '#B5AFDD', '#CAA2D4', '#DA95C9', '#E684BB', '#EB72A6', '#A2A2EB', '#B89AE5', '#CA8DDD', '#D87DD5', '#E16BC2', '#C27AE8', '#D266DB', '#AF87F0', '#BD65ED', '#8B9FF0', '#988DF5', '#AB72F6', '#A95BF6', '#9E48F2', '#8B7BFC', '#9668FC', '#8A54FB', '#98FB59', '#B0F455', '#C5E84D', '#D5D93F', '#8BFC76', '#A3F775', '#B8EF72', '#CCE16A', '#D9D15D', '#97F592', '#AEEF8F', '#C4E38A', '#D5D581', '#E0C374', '#8BEDAC', '#A2EAAB', '#BAE1A8', '#CCD3A1', '#DCC493', '#E7B080', '#93E0C4', '#ACD9C0', '#C0CEBB', '#D2C0B2', '#E3B0A0', '#EE9D8B', '#9ECDD3', '#B3C4D1', '#C6B7CB', '#D8ABBE', '#E79DAD', '#EF8C9E', '#F17989', '#98077D', '#B00F7B', '#C61A7A', '#D92A78', '#8D0161', '#A50761', '#BC115E', '#D01F5D', '#980344', '#B00A43', '#C61642', '#D92741', '#8B0129', '#A40527', '#BC0F26', '#D01D25', '#97020B', '#B0090A', '#C6150A', '#9236E7', '#B54AEA', '#8A26D3', '#AB37E0', '#C953DD', '#951BBB', '#A727CB', '#BE31CC', '#D142CA', '#DB56C7', '#AD1BB0', '#C126B0', '#D333AF', '#E048AF', '#E75CAC', '#8A0D9C', '#A41096', '#BA1996', '#CE2695', '#DF3693', '#E94C92', '#EE628F', '#E8B461', '#EFA06B', '#F28B76', '#F36B71', '#F35475', '#E73D76', '#E2C249', '#EEA44B', '#F58F56', '#FB7958', '#FC5D58', '#F54559', '#E1325B', '#DFC92B', '#E9B334', '#F69437', '#FC7D39', '#FD673B', '#FB503D', '#EB3A3F', '#E7B915', '#F0A31F', '#FC881C', '#FD711E', '#FD5A20', '#F54223', '#E03024', '#EEA900', '#F99300', '#FE7C01', '#FF6403', '#FB4C06', '#EA3709', '#D9250A', '#7EB1E8', '#7E8DF7', '#7E67FF', '#7E40F2', '#7FFF5A', '#7FF993', '#7FD7CE', '#7E0682', '#7E0144', '#7E000E', '#7E15B4'], interactions={'hover': 'tooltip'}, scales={'color': ColorScale(colors=['orangered', 'black', 'deepskyblue'], max=277.79314281954834, min=-295.89795148495813), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([-0.04894027, -0.06781295, -0.0823638 , -0.09256381, -0.09657028,
     

In [16]:
def update_colors(name, value):
    sel = l.selected
    if len(sel) == 0:
        return
    scat.color = data_mean[:, sel[0]] * 1e15
    fig2.title = 'Snapshot at t = {:.1f} ms'.format(value[0])
    
fig = plt.figure(title='MEG signal', layout={'min_width': '800px'})

l = plt.plot(1e3 * epochs.times, 1e15 * data_mean,
         stroke_width=.2, interpolation='cardinal',
         colors=colors,
         axes_options={'x': {'label': 'Time (ms)'},
                       'y': {'label': 'Magnetic Field (fT)'}})
index_sel = plt.index_selector(update_colors)
plt.vline(0, line_style='dashed')

display(fig, fig2)

Figure(axes=[Axis(label='Time (ms)', scale=LinearScale()), Axis(label='Magnetic Field (fT)', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=IndexSelector(marks=[Lines(colors=['#48AFDC', '#33A3D3', '#2395C8', '#1884BB', '#1372A5', '#5BA3EA', '#459AE4', '#338DDC', '#267DD4', '#1D6BC1', '#3B7AE7', '#2B67DA', '#4E87EF', '#4065EC', '#729FF0', '#658DF4', '#5272F5', '#545BF5', '#5F48F2', '#717BFC', '#6668FC', '#7254FB', '#66FC58', '#4EF455', '#39E84D', '#29D93F', '#72FC76', '#5BF775', '#45EF72', '#31E16A', '#24D15C', '#66F592', '#4FEF8F', '#39E38A', '#29D581', '#1EC374', '#72EDAC', '#5BEAAB', '#43E0A7', '#32D3A1', '#21C493', '#16B080', '#6AE0C4', '#51D9C0', '#3CCEBB', '#2BC0B2', '#1BB09F', '#109D8B', '#5FCDD2', '#4AC4D1', '#37B7CA', '#26ABBD', '#179DAC', '#0F8C9D', '#0D7989', '#64087C', '#4C0F7B', '#361B79', '#242B78', '#700261', '#580860', '#40125E', '#2D205D', '#640344', '#4C0B43', '#361742', '#242841', '#700129', '#411026', '#2D1E26', '#64020B', '#4C090A', '#37160A', '#6A36E7', '#474AE9', '#7226D3', '#5238DF', '#3453DC', '#671BBB', '#5527CB', '#3E32CC', '#2C43C9', '#2357C6', '#4F1BAF', '#3C26B0', '#2934AF', '#1D48AF', '#175DAB', '#720D9B', '#591196', '#431995', '#2E2695', '#1F3792', '#144D92', '#10628E', '#16B460', '#0FA06B', '#0C8B75', '#0C6C71', '#0B5575', '#173E76', '#1CC249', '#0FA44B', '#098F56', '#047958', '#025D58', '#094659', '#1D335B', '#1EC92B', '#15B334', '#099437', '#027E39', '#01673B', '#04503D', '#133A3F', '#16BA16', '#0EA31F', '#02881C', '#01711E', '#015A20', '#084323', '#1D3125', '#0FA900', '#069300', '#007C01', '#006403', '#034D06', '#143808', '#24260A', '#8AC3DE', '#9FB7E1', '#B5AFDD', '#CAA2D4', '#DA95C9', '#E684BB', '#EB72A6', '#A2A2EB', '#B89AE5', '#CA8DDD', '#D87DD5', '#E16BC2', '#C27AE8', '#D266DB', '#AF87F0', '#BD65ED', '#8B9FF0', '#988DF5', '#AB72F6', '#A95BF6', '#9E48F2', '#8B7BFC', '#9668FC', '#8A54FB', '#98FB59', '#B0F455', '#C5E84D', '#D5D93F', '#8BFC76', '#A3F775', '#B8EF72', '#CCE16A', '#D9D15D', '#97F592', '#AEEF8F', '#C4E38A', '#D5D581', '#E0C374', '#8BEDAC', '#A2EAAB', '#BAE1A8', '#CCD3A1', '#DCC493', '#E7B080', '#93E0C4', '#ACD9C0', '#C0CEBB', '#D2C0B2', '#E3B0A0', '#EE9D8B', '#9ECDD3', '#B3C4D1', '#C6B7CB', '#D8ABBE', '#E79DAD', '#EF8C9E', '#F17989', '#98077D', '#B00F7B', '#C61A7A', '#D92A78', '#8D0161', '#A50761', '#BC115E', '#D01F5D', '#980344', '#B00A43', '#C61642', '#D92741', '#8B0129', '#A40527', '#BC0F26', '#D01D25', '#97020B', '#B0090A', '#C6150A', '#9236E7', '#B54AEA', '#8A26D3', '#AB37E0', '#C953DD', '#951BBB', '#A727CB', '#BE31CC', '#D142CA', '#DB56C7', '#AD1BB0', '#C126B0', '#D333AF', '#E048AF', '#E75CAC', '#8A0D9C', '#A41096', '#BA1996', '#CE2695', '#DF3693', '#E94C92', '#EE628F', '#E8B461', '#EFA06B', '#F28B76', '#F36B71', '#F35475', '#E73D76', '#E2C249', '#EEA44B', '#F58F56', '#FB7958', '#FC5D58', '#F54559', '#E1325B', '#DFC92B', '#E9B334', '#F69437', '#FC7D39', '#FD673B', '#FB503D', '#EB3A3F', '#E7B915', '#F0A31F', '#FC881C', '#FD711E', '#FD5A20', '#F54223', '#E03024', '#EEA900', '#F99300', '#FE7C01', '#FF6403', '#FB4C06', '#EA3709', '#D9250A', '#7EB1E8', '#7E8DF7', '#7E67FF', '#7E40F2', '#7FFF5A', '#7FF993', '#7FD7CE', '#7E0682', '#7E0144', '#7E000E', '#7E15B4'], interactions={'hover': 'tooltip'}, interpolation='cardinal', scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, stroke_width=0.2, tooltip_style={'opacity': 0.9}, x=array([-100.        ,  -95.83333333,  -91.66666667,  -87.5       ,
        -83.33333333,  -79.16666667,  -75.        ,  -70.83333333,
        -66.66666667,  -62.5       ,  -58.33333333,  -54.16666667,
        -50.        ,  -45.83333333,  -41.66666667,  -37.5       ,
        -33.33333333,  -29.16666667,  -25.        ,  -20.83333333,
        -16.66666667,  -12.5       ,   -8.33333333,   -4.16666667,
          0.        ,    4.16666667,

Figure(axes=[ColorAxis(scale=ColorScale(colors=['orangered', 'black', 'deepskyblue'], max=277.79314281954834, min=-295.89795148495813)), Axis(scale=LinearScale(), side='bottom', visible=False), Axis(orientation='vertical', scale=LinearScale(), side='left', visible=False)], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_height='600px', min_width='600px'), marks=[Scatter(colors=['#48AFDC', '#33A3D3', '#2395C8', '#1884BB', '#1372A5', '#5BA3EA', '#459AE4', '#338DDC', '#267DD4', '#1D6BC1', '#3B7AE7', '#2B67DA', '#4E87EF', '#4065EC', '#729FF0', '#658DF4', '#5272F5', '#545BF5', '#5F48F2', '#717BFC', '#6668FC', '#7254FB', '#66FC58', '#4EF455', '#39E84D', '#29D93F', '#72FC76', '#5BF775', '#45EF72', '#31E16A', '#24D15C', '#66F592', '#4FEF8F', '#39E38A', '#29D581', '#1EC374', '#72EDAC', '#5BEAAB', '#43E0A7', '#32D3A1', '#21C493', '#16B080', '#6AE0C4', '#51D9C0', '#3CCEBB', '#2BC0B2', '#1BB09F', '#109D8B', '#5FCDD2', '#4AC4D1', '#37B7CA', '#26ABBD', '#179DAC', '#0F8C9D', '#0D7989', '#64087C', '#4C0F7B', '#361B79', '#242B78', '#700261', '#580860', '#40125E', '#2D205D', '#640344', '#4C0B43', '#361742', '#242841', '#700129', '#411026', '#2D1E26', '#64020B', '#4C090A', '#37160A', '#6A36E7', '#474AE9', '#7226D3', '#5238DF', '#3453DC', '#671BBB', '#5527CB', '#3E32CC', '#2C43C9', '#2357C6', '#4F1BAF', '#3C26B0', '#2934AF', '#1D48AF', '#175DAB', '#720D9B', '#591196', '#431995', '#2E2695', '#1F3792', '#144D92', '#10628E', '#16B460', '#0FA06B', '#0C8B75', '#0C6C71', '#0B5575', '#173E76', '#1CC249', '#0FA44B', '#098F56', '#047958', '#025D58', '#094659', '#1D335B', '#1EC92B', '#15B334', '#099437', '#027E39', '#01673B', '#04503D', '#133A3F', '#16BA16', '#0EA31F', '#02881C', '#01711E', '#015A20', '#084323', '#1D3125', '#0FA900', '#069300', '#007C01', '#006403', '#034D06', '#143808', '#24260A', '#8AC3DE', '#9FB7E1', '#B5AFDD', '#CAA2D4', '#DA95C9', '#E684BB', '#EB72A6', '#A2A2EB', '#B89AE5', '#CA8DDD', '#D87DD5', '#E16BC2', '#C27AE8', '#D266DB', '#AF87F0', '#BD65ED', '#8B9FF0', '#988DF5', '#AB72F6', '#A95BF6', '#9E48F2', '#8B7BFC', '#9668FC', '#8A54FB', '#98FB59', '#B0F455', '#C5E84D', '#D5D93F', '#8BFC76', '#A3F775', '#B8EF72', '#CCE16A', '#D9D15D', '#97F592', '#AEEF8F', '#C4E38A', '#D5D581', '#E0C374', '#8BEDAC', '#A2EAAB', '#BAE1A8', '#CCD3A1', '#DCC493', '#E7B080', '#93E0C4', '#ACD9C0', '#C0CEBB', '#D2C0B2', '#E3B0A0', '#EE9D8B', '#9ECDD3', '#B3C4D1', '#C6B7CB', '#D8ABBE', '#E79DAD', '#EF8C9E', '#F17989', '#98077D', '#B00F7B', '#C61A7A', '#D92A78', '#8D0161', '#A50761', '#BC115E', '#D01F5D', '#980344', '#B00A43', '#C61642', '#D92741', '#8B0129', '#A40527', '#BC0F26', '#D01D25', '#97020B', '#B0090A', '#C6150A', '#9236E7', '#B54AEA', '#8A26D3', '#AB37E0', '#C953DD', '#951BBB', '#A727CB', '#BE31CC', '#D142CA', '#DB56C7', '#AD1BB0', '#C126B0', '#D333AF', '#E048AF', '#E75CAC', '#8A0D9C', '#A41096', '#BA1996', '#CE2695', '#DF3693', '#E94C92', '#EE628F', '#E8B461', '#EFA06B', '#F28B76', '#F36B71', '#F35475', '#E73D76', '#E2C249', '#EEA44B', '#F58F56', '#FB7958', '#FC5D58', '#F54559', '#E1325B', '#DFC92B', '#E9B334', '#F69437', '#FC7D39', '#FD673B', '#FB503D', '#EB3A3F', '#E7B915', '#F0A31F', '#FC881C', '#FD711E', '#FD5A20', '#F54223', '#E03024', '#EEA900', '#F99300', '#FE7C01', '#FF6403', '#FB4C06', '#EA3709', '#D9250A', '#7EB1E8', '#7E8DF7', '#7E67FF', '#7E40F2', '#7FFF5A', '#7FF993', '#7FD7CE', '#7E0682', '#7E0144', '#7E000E', '#7E15B4'], interactions={'hover': 'tooltip'}, scales={'color': ColorScale(colors=['orangered', 'black', 'deepskyblue'], max=277.79314281954834, min=-295.89795148495813), 'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([-0.04894027, -0.06781295, -0.0823638 , -0.09256381, -0.09

## How about 3d?

- pythreeJS https://github.com/jovyan/pythreejs
- ipyvolume https://github.com/maartenbreddels/ipyvolume

3d-plotting libraries based on widgets and WebGL

In [17]:
from pythreejs import *
import numpy as np
from IPython.display import display

sensors = [Mesh(geometry=SphereGeometry(radius=0.003), 
                   material=LambertMaterial(color=colors[i]),
                   position=[x[i], y[i], z[i]])
              for i in range(len(x))]

scene = Scene(children=sensors + [AmbientLight(color='white')])

c = PerspectiveCamera(position=[0, 1, 1], up=[0, 0, 2],)

renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])
# display(renderer)

In [18]:
red, black, blue = np.array([255, 69, 0])/255, np.array([.2, .2, .2]), np.array([0, 191, 255])/255
col_sc = diverging_color_scale(data_mean, [red, black, blue])
precomputed_colors = np.array([[rgb_to_css(col_sc(d)) for d in di] for di in data_mean])

def update_colors_3d(change):
    sel = l.selected
    if len(sel) == 0:
        return
    for i, b in enumerate(sensors):
        b.material.color = precomputed_colors[i, sel[0]]
        
fig = plt.figure(title='MEG signal', layout={'min_width': '800px'})

l = plt.plot(1e3 * epochs.times, 1e15 * data_mean,
         stroke_width=.2, interpolation='cardinal',
         colors=colors,
         axes_options={'x': {'label': 'Time (ms)'},
                       'y': {'label': 'Magnetic Field (fT)'}})
plt.index_selector(update_colors_3d)
plt.vline(0, line_style='dashed')

display(fig, renderer)

Figure(axes=[Axis(label='Time (ms)', scale=LinearScale()), Axis(label='Magnetic Field (fT)', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=IndexSelector(marks=[Lines(colors=['#48AFDC', '#33A3D3', '#2395C8', '#1884BB', '#1372A5', '#5BA3EA', '#459AE4', '#338DDC', '#267DD4', '#1D6BC1', '#3B7AE7', '#2B67DA', '#4E87EF', '#4065EC', '#729FF0', '#658DF4', '#5272F5', '#545BF5', '#5F48F2', '#717BFC', '#6668FC', '#7254FB', '#66FC58', '#4EF455', '#39E84D', '#29D93F', '#72FC76', '#5BF775', '#45EF72', '#31E16A', '#24D15C', '#66F592', '#4FEF8F', '#39E38A', '#29D581', '#1EC374', '#72EDAC', '#5BEAAB', '#43E0A7', '#32D3A1', '#21C493', '#16B080', '#6AE0C4', '#51D9C0', '#3CCEBB', '#2BC0B2', '#1BB09F', '#109D8B', '#5FCDD2', '#4AC4D1', '#37B7CA', '#26ABBD', '#179DAC', '#0F8C9D', '#0D7989', '#64087C', '#4C0F7B', '#361B79', '#242B78', '#700261', '#580860', '#40125E', '#2D205D', '#640344', '#4C0B43', '#361742', '#242841', '#700129', '#411026', '#2D1E26', '#64020B', '#4C090A', '#37160A', '#6A36E7', '#474AE9', '#7226D3', '#5238DF', '#3453DC', '#671BBB', '#5527CB', '#3E32CC', '#2C43C9', '#2357C6', '#4F1BAF', '#3C26B0', '#2934AF', '#1D48AF', '#175DAB', '#720D9B', '#591196', '#431995', '#2E2695', '#1F3792', '#144D92', '#10628E', '#16B460', '#0FA06B', '#0C8B75', '#0C6C71', '#0B5575', '#173E76', '#1CC249', '#0FA44B', '#098F56', '#047958', '#025D58', '#094659', '#1D335B', '#1EC92B', '#15B334', '#099437', '#027E39', '#01673B', '#04503D', '#133A3F', '#16BA16', '#0EA31F', '#02881C', '#01711E', '#015A20', '#084323', '#1D3125', '#0FA900', '#069300', '#007C01', '#006403', '#034D06', '#143808', '#24260A', '#8AC3DE', '#9FB7E1', '#B5AFDD', '#CAA2D4', '#DA95C9', '#E684BB', '#EB72A6', '#A2A2EB', '#B89AE5', '#CA8DDD', '#D87DD5', '#E16BC2', '#C27AE8', '#D266DB', '#AF87F0', '#BD65ED', '#8B9FF0', '#988DF5', '#AB72F6', '#A95BF6', '#9E48F2', '#8B7BFC', '#9668FC', '#8A54FB', '#98FB59', '#B0F455', '#C5E84D', '#D5D93F', '#8BFC76', '#A3F775', '#B8EF72', '#CCE16A', '#D9D15D', '#97F592', '#AEEF8F', '#C4E38A', '#D5D581', '#E0C374', '#8BEDAC', '#A2EAAB', '#BAE1A8', '#CCD3A1', '#DCC493', '#E7B080', '#93E0C4', '#ACD9C0', '#C0CEBB', '#D2C0B2', '#E3B0A0', '#EE9D8B', '#9ECDD3', '#B3C4D1', '#C6B7CB', '#D8ABBE', '#E79DAD', '#EF8C9E', '#F17989', '#98077D', '#B00F7B', '#C61A7A', '#D92A78', '#8D0161', '#A50761', '#BC115E', '#D01F5D', '#980344', '#B00A43', '#C61642', '#D92741', '#8B0129', '#A40527', '#BC0F26', '#D01D25', '#97020B', '#B0090A', '#C6150A', '#9236E7', '#B54AEA', '#8A26D3', '#AB37E0', '#C953DD', '#951BBB', '#A727CB', '#BE31CC', '#D142CA', '#DB56C7', '#AD1BB0', '#C126B0', '#D333AF', '#E048AF', '#E75CAC', '#8A0D9C', '#A41096', '#BA1996', '#CE2695', '#DF3693', '#E94C92', '#EE628F', '#E8B461', '#EFA06B', '#F28B76', '#F36B71', '#F35475', '#E73D76', '#E2C249', '#EEA44B', '#F58F56', '#FB7958', '#FC5D58', '#F54559', '#E1325B', '#DFC92B', '#E9B334', '#F69437', '#FC7D39', '#FD673B', '#FB503D', '#EB3A3F', '#E7B915', '#F0A31F', '#FC881C', '#FD711E', '#FD5A20', '#F54223', '#E03024', '#EEA900', '#F99300', '#FE7C01', '#FF6403', '#FB4C06', '#EA3709', '#D9250A', '#7EB1E8', '#7E8DF7', '#7E67FF', '#7E40F2', '#7FFF5A', '#7FF993', '#7FD7CE', '#7E0682', '#7E0144', '#7E000E', '#7E15B4'], interactions={'hover': 'tooltip'}, interpolation='cardinal', scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, stroke_width=0.2, tooltip_style={'opacity': 0.9}, x=array([-100.        ,  -95.83333333,  -91.66666667,  -87.5       ,
        -83.33333333,  -79.16666667,  -75.        ,  -70.83333333,
        -66.66666667,  -62.5       ,  -58.33333333,  -54.16666667,
        -50.        ,  -45.83333333,  -41.66666667,  -37.5       ,
        -33.33333333,  -29.16666667,  -25.        ,  -20.83333333,
        -16.66666667,  -12.5       ,   -8.33333333,   -4.16666667,
          0.        ,    4.16666667,

Renderer(camera=PerspectiveCamera(position=[0.0, 1.0, 1.0], scale=[1.0, 1.0, 1.0], up=[0.0, 0.0, 2.0]), controls=[OrbitControls(controlling=PerspectiveCamera(position=[0.0, 1.0, 1.0], scale=[1.0, 1.0, 1.0], up=[0.0, 0.0, 2.0]), target=[0.0, 0.0, 0.0])], effect=None, scene=Scene(children=[Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#48AFDC', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.048940274539483325, 0.046760971439866385, 0.07705000000000001], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#33A3D3', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.06781295452935228, 0.03431872051810791, 0.070854], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#2395C8', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.08236379787260906, 0.020712571834516344, 0.06323000000000001], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#1884BB', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.09256381319122502, 0.00399586042048518, 0.054394], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#1372A5', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.09657028021342799, -0.014193754418757571, 0.040090000000000015], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#5BA3EA', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.03241660327671609, 0.03432154894523265, 0.086215], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#459AE4', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.05173193211160782, 0.02606678438166089, 0.082439], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#338DDC', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.06783982458703737, 0.013360075523738624, 0.076709], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#267DD4', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.07977649416024749, -0.002872974851960948, 0.071151], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#1D6BC1', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.08796337647282533, -0.02058175707999685, 0.05899300000000002], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#3B7AE7', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.060903814170378524, -0.005749485237827816, 0.084315], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#2B67DA', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.07507069853145101, -0.02503299426756616, 0.075763], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#4E87EF', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.044001840779676483, 0.007164405906982097, 0.089787], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#4065EC', envMap=None, lightMap=None, map=None, specularMap=None), position=[-0.05664420292051076, -0.02669115966944861, 0.087498], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), Mesh(geometry=SphereGeometry(radius=0.003), material=LambertMaterial(color='#729FF0', envMap=None, lightMap=None, map=None, specularMap=